## Grouping and summarizing by Centreline

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import time
from functools import reduce

In [2]:
years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

dfs_all = []
dfs_load = []
dfs_bike = []

load_string = "PARK COMMERC LOAD ZONE NOT LDG|PARK VEH.-COMMERCIAL LOAD ZONE|PARK-PASSENGER/FREIGHT LOADING|PARK PASSENGER/FREIGHT LOAD ZO"
bike_string = ""

for year in years:
    
    print(year)
    df = pd.read_csv("data/tickets-with-centreline/twc-" + str(year) + ".gz")
    
    # summary all
    dfc = df.groupby(['CENTRELINE_ID']).size().reset_index(name='counts')
    dfd = df.groupby(['CENTRELINE_ID'])['set_fine_amount'].sum().reset_index()
    dfs = pd.merge(dfc,dfd,on="CENTRELINE_ID")
    dfs.columns=["CENTRELINE_ID","count-" + str(year), "fines-" + str(year)]
    dfs_all.append(dfs)
    
    # summary loading
    dfc = df[df["infraction_description"].str.contains(load_string)].groupby(['CENTRELINE_ID']).size().reset_index(name='counts')
    dfd = df[df["infraction_description"].str.contains(load_string)].groupby(['CENTRELINE_ID'])['set_fine_amount'].sum().reset_index()
    dfs = pd.merge(dfc,dfd,on="CENTRELINE_ID")
    dfs.columns=["CENTRELINE_ID","count-" + str(year), "fines-" + str(year)]
    dfs_load.append(dfs)


# all table
dfs_all = reduce(lambda left,right: pd.merge(left,right,on=['CENTRELINE_ID'],how='outer'), dfs_all)
dfs_all = dfs_all.fillna(0)
dfs_all["count_all"] = 0
dfs_all["fines_all"] = 0
for year in years:
    dfs_all["count_all"] = dfs_all["count_all"] + dfs_all["count-" + str(year)]
    dfs_all["fines_all"] = dfs_all["fines_all"] + dfs_all["fines-" + str(year)]
dfs_all.to_csv("data/centreline-stats/all.csv", index=False)

# all table
dfs_load = reduce(lambda left,right: pd.merge(left,right,on=['CENTRELINE_ID'],how='outer'), dfs_load)
dfs_load = dfs_load.fillna(0)
dfs_load["count_all"] = 0
dfs_load["fines_all"] = 0
for year in years:
    dfs_load["count_all"] = dfs_load["count_all"] + dfs_load["count-" + str(year)]
    dfs_load["fines_all"] = dfs_load["fines_all"] + dfs_load["fines-" + str(year)]
dfs_load.to_csv("data/centreline-stats/load.csv", index=False)


2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [3]:
dfs_all

,CENTRELINE_ID,count-2011,fines-2011,count-2012,fines-2012,count-2013,fines-2013,count-2014,fines-2014,count-2015,...,count-2017,fines-2017,count-2018,fines-2018,count-2019,fines-2019,count-2020,fines-2020,count_all,fines_all
0,-2,10623.0,421665.0,9861.0,389055.0,10831.0,432205.0,10874.0,469325.0,8681.0,...,8219.0,403725.0,7977.0,378840.0,3865.0,159875.0,3838.0,151930.0,82953.0,3599460.0
1,-1,5336.0,228030.0,6253.0,262320.0,6716.0,275940.0,5356.0,229650.0,4987.0,...,4576.0,217980.0,4193.0,201575.0,5874.0,300215.0,3142.0,171790.0,51488.0,2368725.0
2,0,154719.0,6161355.0,148731.0,5835550.0,116978.0,4670960.0,111766.0,4642365.0,103236.0,...,86423.0,3831210.0,90243.0,4009265.0,104788.0,4764045.0,69038.0,3031480.0,1072370.0,44909955.0
3,117,24.0,935.0,14.0,510.0,35.0,1325.0,69.0,2820.0,28.0,...,7.0,315.0,5.0,205.0,16.0,735.0,1.0,30.0,206.0,8070.0
4,120,8.0,245.0,1.0,40.0,8.0,320.0,4.0,110.0,6.0,...,29.0,535.0,7.0,170.0,18.0,405.0,7.0,220.0,94.0,2225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37651,30121156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,500.0,2.0,500.0
37652,30123367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0,1.0,100.0
37653,30123696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1.0,50.0
37654,30127036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,1.0,30.0


In [4]:
dfs_load

,CENTRELINE_ID,count-2011,fines-2011,count-2012,fines-2012,count-2013,fines-2013,count-2014,fines-2014,count-2015,...,count-2017,fines-2017,count-2018,fines-2018,count-2019,fines-2019,count-2020,fines-2020,count_all,fines_all
0,-2,30.0,1200.0,28.0,1120.0,27.0,1080.0,16.0,640.0,20.0,...,42.0,1680.0,26.0,1040.0,2.0,80.0,0.0,0.0,200.0,8000.0
1,-1,5.0,200.0,2.0,80.0,10.0,400.0,11.0,440.0,6.0,...,18.0,720.0,15.0,600.0,22.0,880.0,9.0,360.0,114.0,4560.0
2,0,880.0,35200.0,687.0,27480.0,510.0,20400.0,483.0,19200.0,351.0,...,406.0,16200.0,671.0,26840.0,581.0,23240.0,348.0,13920.0,5106.0,203680.0
3,1579,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0
4,437941,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,30084062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,80.0,2.0,80.0
583,30095067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,1.0,40.0
584,30100056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,1.0,40.0
585,30106817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,1.0,40.0
